# LMPerf Testing

## Machine requirements

This notebook should be run on a machine with at least 1 Nvidia GPU, or 1 AWS AI chip. Below we show recommended machines for a range of LLM sizes. You can try smaller machines to see if they work for your tested model.

**Note:** As LLM size grows bigger, it may not fit on a single machine, and cannot be tested using this notebook.

### For Nvidia GPUs

| LLM size      | EC2 instance type |
| ----------- | ----------- |
| Upto 10B      | [g6.48xlarge](https://aws.amazon.com/ec2/instance-types/g6/), [g5.48xlarge](https://aws.amazon.com/ec2/instance-types/g5/)      |
| Upto 70B   | [g6e.48xlarge](https://aws.amazon.com/ec2/instance-types/g6e/), [p4d.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| Upto 405B   | [p5.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/), [p4de.24xlarge](https://aws.amazon.com/ec2/instance-types/p4/)      |
| > 405B   | [p5e.48xlarge](https://aws.amazon.com/ec2/instance-types/p5/)      |

### For AWS AI Chips

| LLM size      | EC2 instance type |
| ----------- | ----------- |    
| Upto 70B   | [trn1.32xlarge](https://aws.amazon.com/ec2/instance-types/trn1/)       |
| > 70B   | [trn2.48xlarge](https://aws.amazon.com/ec2/instance-types/trn2/)      |

Next, let us verify the machine.


In [ ]:
import sys
sys.dont_write_bytecode = True

import os
import subprocess
p = subprocess.run('nvidia-smi --list-gpus | wc -l', 
                   shell=True, check=True, capture_output=True, encoding='utf-8')

device = None
num_device = 0

if p.returncode == 0:
    num_device = int(p.stdout)
    device = "cuda" if num_device > 0 else None

if device is None:
    p = subprocess.run('neuron-ls -j | grep neuron_device | wc -l', 
                       shell=True, check=True, capture_output=True, encoding='utf-8')
    if p.returncode == 0:
        num_device = int(p.stdout)
        device = "neuron" if num_device > 0 else None

assert (device == "cuda" and num_device >= 1) or (device == "neuron" and num_device >= 1), \
    "Machine must have 1 Nvidia CUDA devices, or 1 AWS Neuorn Devices"
print(f"Auto detected {num_device} {device} devices")
os.environ['NUM_DEVICE']=str(num_device)
os.environ['DEVICE']=device

if device == 'neuron':
    p = subprocess.run("neuron-ls --json-output | grep nc_count | head -1 | awk -F': ' '{print $2}' | tr -d ','",
                       shell=True, check=True, capture_output=True, encoding='utf-8')
    if p.returncode == 0:
        neuron_cores_per_device = int(p.stdout)
        print(f"NeuronCores per device: {neuron_cores_per_device}")

Below we install required packages.

In [ ]:
!pip install --upgrade pip
!pip install datasets==3.6.0

### Create Hugging Face User Access Token

Many of the popular Large Language Models (LLMs) in Hugging Face are [gated models](https://huggingface.co/docs/hub/en/models-gated). To access gated models, you need a Hugging Face [user access token](https://huggingface.co/docs/hub/en/security-tokens). Please create a Hugging Face user access token in your Hugging Face account, and set it below in `hf_token` variable below.

In [ ]:
import subprocess
import time
import os
import yaml
import json

hf_token=''
# Comment out next line if not using a Hugging Face gated model
assert hf_token, "Hugging Face user access token is required for gated models"
os.environ['HF_TOKEN']=hf_token

### Build Docker Containers

Next, we build the docker containers used to run the inference endpoint locally on this desktop. 

In [ ]:
! source scripts/build-containers.sh

### Specify Hugging Face Model Id

Next, set the Hugging Face Model Id for the LLM you want to test in `hf_model_id` variable, below. You can specify the Hugging Face model id for a [Meta Llama](https://huggingface.co/meta-llama), [Deepseek](https://huggingface.co/deepseek-ai), or [Mistral AI](https://huggingface.co/mistralai) text-based Generative AI LLM. 

In [ ]:
hf_model_id = 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B'
os.environ['MAX_MODEL_LEN']=str(8192)
decoder_type="text_only"
assert decoder_type in ["text_only", "text_image", "code_gen"]

### Snapshot Huggingface model

Below we snapshot the Huggingface model and store it on the EFS. This is only done once. To force a refresh of the model from Huggingface hub, you must delete the local copy of the model from the EFS.

To use EFS, we create a symbolic link from `/home/ubuntu/snapshots` to `/home/ubuntu/efs/home/snapshots` directory. Please ensure `/home/ubuntu/efs/home` exists and is owned by user `ubuntu`.

In [ ]:
from huggingface_hub import snapshot_download, list_repo_files
from tempfile import TemporaryDirectory
from pathlib import Path
import shutil
import pwd

home_dir = os.path.join(os.getenv('HOME'))
efs_home = os.path.join(home_dir, "efs", "home")

assert os.path.isdir(efs_home), f"{efs_home} directory must exist"

stat_info = os.stat(efs_home)
owner_uid = stat_info.st_uid
owner_username = pwd.getpwuid(owner_uid).pw_name
assert owner_username == "ubuntu", f"{efs_home} must be owned by ubuntu"
efs_snapshots = os.path.join(efs_home, "snapshots")
os.makedirs(efs_snapshots, exist_ok=True)
if not os.path.exists(os.path.join(home_dir, "snapshots")):
    os.symlink(efs_snapshots, os.path.join(home_dir, "snapshots")) # create a symbolic link to EFS directory

hf_home = os.path.join(home_dir, "snapshots", "huggingface")
os.makedirs(hf_home, exist_ok=True)

model_path = os.path.join(hf_home, hf_model_id)

hub_files = set(list_repo_files(repo_id=hf_model_id, repo_type="model"))
# Get local files (recursively)
local_files = set()
missing_files=None
for root, dirs, files in os.walk(model_path):
    for file in files:
        # Get relative path from model root
        local_files.add(os.path.relpath(os.path.join(root, file), model_path))
    
# Compare
missing_files = hub_files - local_files

if missing_files:
    print(f"Downloading HuggingFace model snapshot: {hf_model_id}")
    os.makedirs(model_path, exist_ok=True)
    with TemporaryDirectory(suffix="model", prefix="hf", dir="/tmp") as cache_dir:
        snapshot_download(repo_id=hf_model_id, cache_dir=cache_dir, token=hf_token)
        local_model_path = Path(cache_dir)
        model_snapshot_path = str(list(local_model_path.glob(f"**/snapshots/*/"))[0])
        print(f"Model snapshot: {model_snapshot_path} completed")
        
        print(f"Copying model snapshot files to EFS...")
        for root, dirs, files in os.walk(model_snapshot_path):
            for file in files:
                full_path = os.path.join(root, file)
                relative_path = f"{full_path[len(model_snapshot_path)+1:]}"
                dst_path = os.path.join(model_path, relative_path)
                dst_dir = os.path.dirname(dst_path)
                os.makedirs(dst_dir, exist_ok=True)
                print(f"Copying {os.path.basename(full_path)}")
                shutil.copyfile(full_path, dst_path)


os.environ['MODEL_ID']=model_path[len(home_dir):] # docker container volume mounts snapshots at /snapshots
print(f"MODEL_ID={os.environ['MODEL_ID']}")

## Configure Tensor Parallel Size

Tensor parallel size depends on number of available device cores (i.e. GPUs, or NeuronCores), and model size. We set tensor parallel size, by default, to minimum of number of available cores, or 8. Depending on the size of your model, you may adjust tensor parallel size , down or up, as needed. The number of inference servers is computed by dividing number of cores by tensor parallel size. For example, if there are 8 cores, and tensor parallel size is 2, 4 inference servers will be started.

In [ ]:
num_cores = num_device * neuron_cores_per_device if device == "neuron" else num_device
tp_parallel_size = min(8, num_cores)
while (num_cores % tp_parallel_size) != 0:
    tp_parallel_size = tp_parallel_size // 2

print(f"tp_parallel_size: {tp_parallel_size}")
os.environ['TENSOR_PARALLEL_SIZE']=str(tp_parallel_size)
max_num_seqs=max(tp_parallel_size, 2)
print(f"max_num_seqs: {max_num_seqs}")
os.environ['MAX_NUM_SEQS']=str(max_num_seqs)

### Specify Inference Server and Engine

Next, specify `inference_server`, and `inference_engine` variables, below. This notebook supports [Triton Inference Server](https://github.com/triton-inference-server/server) with [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM), and [vLLM](https://github.com/vllm-project/vllm) as backends. It also supports Open AI compatible inference server with [vLLM](https://github.com/vllm-project/vllm), and this combination does not require LiteLLM proxy endpoint.

In [ ]:
inference_server = 'openai_server'
assert inference_server in ['triton_inference_server', "openai_server", "djl_serving"]
inference_engine = 'vllm'
assert inference_engine in ['vllm', 'trtllm']

assert  inference_engine != 'trtllm' or inference_server == "triton_inference_server", \
    "TensorRT-LLM is only supported with Triton Inference Server"

os.environ["INFERENCE_SERVER"] = inference_server
os.environ["INFERENCE_ENGINE"] = inference_engine
print(f"Using '{inference_server}' inference server with '{inference_engine}' inference engine")

If the inference server is `triton-inference-server` and the inference engine is `trtllm`, we must specify the path to the [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM/releases/tag/v0.21.0) Python script that converts the Hugging Face model weights to TensorRT checkpoint weights for your selected model.

In [ ]:
if inference_server == "triton_inference_server" and inference_engine == 'trtllm':
    # set the convert ckpt script path compatible with your model
    os.environ["TRTLLM_CONVERT_CKPT_SCRIPT"]="/opt/TensorRT-LLM/examples/models/core/llama/convert_checkpoint.py"

### Launch Inference Server

Next we use Docker compose to launch the inference server locally on this desktop.

In [ ]:
script_path = "./launch.sh"
! bash {script_path} down
! bash {script_path} up

## LiteLLM Testing

### Load Configuration

Below. we load the appropriate configuration file for the specified inference server, and backend.

In [ ]:
path = [ "config", "decoder", decoder_type, f"{inference_server}-{inference_engine}.yaml" ]

config_path=os.path.join(*path)
with open(config_path, "r") as mf:
    config=yaml.safe_load(mf)

print(json.dumps(config, indent=2))

### Verify inference server is up

The inference server may take several minutes to start up. Next, we verify the inference server is up. Do not proceed to next cell until inference server is up.

In [ ]:
import requests
try:
    response = requests.get(config['endpoint_url'])
    response_code = int(response.status_code)
    assert (response_code == 405) or (response_code == 424), f"Inference server is not yet up: {response_code}"
    print("Inference server is up!")
except:
    print("Inference server is not yet up")

### Create LiteLLM config file

Below we create the config file for LiteLLM.

In [ ]:
litellm_config_path=os.path.join("config", "litellm", "config.yaml")
with open(litellm_config_path, "r") as mf:
    litellm_config=yaml.safe_load(mf)

litellm_config['model_list'][0]['litellm_params']['model'] = f"custom-endpoint/{hf_model_id}"
litellm_config['model_list'][0]['litellm_params']['api_base'] = config['endpoint_url']

litellm_config['environment_variables']["TEMPLATE"] = json.dumps(config.get('template', {}))
litellm_config['environment_variables']["TEMPLATE_KEYS"] = json.dumps(config.get('template_keys', []))
litellm_config['environment_variables']["CONTENT_TYPE"]="application/json"
litellm_config['environment_variables']["MODEL"]=os.getenv("MODEL_ID")
litellm_config_path = "/tmp/litellm_config.yaml"
with open(litellm_config_path, "w") as outfile:
    yaml.dump(litellm_config, outfile, default_flow_style=False)

print(f"Configuration saved to {litellm_config_path}")
print(json.dumps(litellm_config, indent=2))



### Start LiteLLM Proxy

Below we start the LiteLLM Proxy as a daemoon.

In [ ]:
# Start Docker container as a daemon process
process = subprocess.Popen([
    'docker', 'run',
    '-d',  # Run in detached/daemon mode
    '-v', f'{litellm_config_path}:/app/config.yaml',
    '-v', './custom_endpoint_handler.py:/app/custom_endpoint_handler.py',
    "--network", "host",
    '-p', '4000:4000',
    'ghcr.io/berriai/litellm:main-stable',
    '--config', '/app/config.yaml'
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the container ID
stdout, stderr = process.communicate()

if process.returncode == 0:
    container_id = stdout.strip()
    print(f"Container started successfully with ID: {container_id}")
else:
    print(f"Error starting container: {stderr}")

### Test the LiteLLM Proxy endpoint

Below we test the LiteLLM Proxy endpoint

In [ ]:
import requests

os.environ["MODEL"] = os.environ['MODEL_ID']
url = "http://localhost:4000/v1/chat/completions"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model": litellm_config['model_list'][0]['litellm_params']['model'],
    "messages": [{"content": "Explain LLM inference to a novice in generative AI."}]
}

response = requests.post(url, headers=headers, json=data)

# Print the response
print(response.json())

### Remove LLMLite Docker container

Below we remove the LLMLite Docker container

In [ ]:
process = subprocess.Popen([
        'docker', 'rm', '-f', container_id  # -f forces removal (stops if running)
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
stdout, stderr = process.communicate()

if process.returncode == 0:
    print(f"Container {container_id} stopped and removed successfully")
else:
    print(f"Error: {stderr}")

### Shutdown LiteLLM proxy and inference server

Next, we shutdown LiteLLM proxy and the inference server.

In [ ]:
! bash {script_path} down